In [12]:
# In this notebook, I'll show how to load Bitcoin data from Binance, convert this into
# a Pandas data frame, calculate exponential moving averages, and visualize the results
# in a candlestick graph using Plotly.


import pandas as pd
import numpy as np
from tzlocal import get_localzone
import requests
import json
import talib
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.core.display import display, HTML

# Notebook settings
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)
pd.set_option('mode.chained_assignment', None)
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
mytz = get_localzone()

# EMA's configurations
config = {
    "EMA1": 7,
    "EMA2": 25,
    "EMA3": 99,
}

def loadDFfromBinance(coinPair, timeInterval):
    resp = requests.get('https://www.binance.com/api/v3/klines?symbol=' + coinPair + '&interval=' + timeInterval)
    df = pd.DataFrame(json.loads(resp.text))
    df = df[[0, 1, 2, 3, 4, 5]]
    df = df.rename(columns = { 0: 'Date', 1: 'Open', 2: 'High', 3: 'Low', 4: 'Close', 5: 'Volume' })
    df = df.set_index(pd.DatetimeIndex(df['Date']))

    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df["Close"] = df['Close'].astype(float)
    df['Open'] = df['Open'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    df['Date'] =  pd.to_datetime(df['Date'], unit='ms').dt.tz_localize('UTC').dt.tz_convert(mytz).dt.strftime('%m/%d/%Y %H:%M')

    return df

def calculateEMAs(df):
    df_copy = df.copy()

    df_copy['EMA1'] = df_copy['Close'].ewm(span=config['EMA1'], adjust=False).mean()
    df_copy['EMA2'] = df_copy['Close'].ewm(span=config['EMA2'], adjust=False).mean()
    df_copy['EMA3'] = df_copy['Close'].ewm(span=config['EMA3'], adjust=False).mean()
    
    return df_copy

def showGraph(df):
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, specs=[[{ 'type': 'candlestick', 'rowspan': 1 }]])
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.update_layout(autosize=False, width=800, height=600)
    fig.add_trace(go.Candlestick(x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'], close=df['Close']), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA1'], line=dict(color='yellow', width=1), name="EMA1"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA2'], line=dict(color='red', width=1), name="EMA2"), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA3'], line=dict(color='green', width=1), name="EMA3"), row=1, col=1)
    fig.show()

showGraph(calculateEMAs(loadDFfromBinance('BTCUSDT', '1h')))



